In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
import yaml

In [5]:
files_name = []
dir_path = 'D:\cricket_score_predictor\T20_data'

for file in os.listdir(dir_path):
    # Get the full file path
    full_path = os.path.join(dir_path, file)
    # Check if the file is not a .txt file since there is one README.txt in the folder
    if not file.endswith('.txt'):
        # Add the file path to the list
        files_name.append(full_path)

In [6]:
files_name[0:5]

['D:\\cricket_score_predictor\\T20_data\\1001349.yaml',
 'D:\\cricket_score_predictor\\T20_data\\1001351.yaml',
 'D:\\cricket_score_predictor\\T20_data\\1001353.yaml',
 'D:\\cricket_score_predictor\\T20_data\\1004729.yaml',
 'D:\\cricket_score_predictor\\T20_data\\1007655.yaml']

In [7]:
final_df=pd.DataFrame()
counter=1
for i in tqdm(files_name):
    with open(i,'r') as f:
        data = yaml.safe_load(f)
        if isinstance(data, list) or isinstance(data, dict):
            df = pd.json_normalize(data)
        else:
            raise ValueError(f"File {i} does not contain a list or dict")
        df['match_id']=counter
        final_df=pd.concat([final_df,df],ignore_index=True)
        counter+=1
final_df

100%|██████████| 1422/1422 [04:36<00:00,  5.13it/s]


,innings,meta.data_version,meta.created,meta.revision,info.dates,info.gender,info.match_type,info.outcome.by.wickets,info.outcome.winner,info.overs,...,match_id,info.city,info.outcome.by.runs,info.match_type_number,info.neutral_venue,info.outcome.method,info.outcome.result,info.outcome.eliminator,info.bowl_out,info.outcome.bowl_out
0,"[{'1st innings': {'team': 'Australia', 'delive...",0.9,2017-02-18,2,[2017-02-17],male,T20,5.0,Sri Lanka,20,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'1st innings': {'team': 'Australia', 'delive...",0.9,2017-02-19,2,[2017-02-19],male,T20,2.0,Sri Lanka,20,...,2,Victoria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'1st innings': {'team': 'Australia', 'delive...",0.9,2017-02-23,1,[2017-02-22],male,T20,NaN,Australia,20,...,3,NaN,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'1st innings': {'team': 'Hong Kong', 'delive...",0.9,2016-09-12,1,[2016-09-05],male,T20,NaN,Hong Kong,20,...,4,Londonderry,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'1st innings': {'team': 'Zimbabwe', 'deliver...",0.9,2016-06-19,1,[2016-06-18],male,T20,NaN,Zimbabwe,20,...,5,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,"[{'1st innings': {'team': 'Sri Lanka', 'delive...",0.9,2016-03-05,2,[2016-03-04],male,T20,6.0,Pakistan,20,...,1418,Mirpur,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1418,"[{'1st innings': {'team': 'Bangladesh', 'deliv...",0.9,2016-03-08,1,[2016-03-06],male,T20,8.0,India,20,...,1419,Mirpur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1419,"[{'1st innings': {'team': 'Netherlands', 'deli...",0.9,2016-02-03,1,[2016-02-03],male,T20,NaN,Netherlands,20,...,1420,Dubai,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,"[{'1st innings': {'team': 'Australia', 'delive...",0.9,2016-09-12,1,[2016-09-06],male,T20,NaN,Australia,20,...,1421,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
final_df.shape

(1422, 26)

Removing columns which are not useful

In [9]:
final_df.columns

Index(['innings', 'meta.data_version', 'meta.created', 'meta.revision',
       'info.dates', 'info.gender', 'info.match_type',
       'info.outcome.by.wickets', 'info.outcome.winner', 'info.overs',
       'info.player_of_match', 'info.teams', 'info.toss.decision',
       'info.toss.winner', 'info.umpires', 'info.venue', 'match_id',
       'info.city', 'info.outcome.by.runs', 'info.match_type_number',
       'info.neutral_venue', 'info.outcome.method', 'info.outcome.result',
       'info.outcome.eliminator', 'info.bowl_out', 'info.outcome.bowl_out'],
      dtype='object')

In [10]:
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.outcome.bowl_out',
    'info.bowl_out',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets'
],inplace=True)

In [11]:
final_df['info.gender'].value_counts()

info.gender
male      956
female    466
Name: count, dtype: int64

In [12]:
final_df=final_df[final_df['info.gender']=='male']
final_df.drop(columns=['info.gender'],inplace=True)
final_df

,innings,info.dates,info.match_type,info.outcome.winner,info.overs,info.player_of_match,info.teams,info.toss.decision,info.toss.winner,info.umpires,info.venue,match_id,info.city
0,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-17],T20,Sri Lanka,20,[DAS Gunaratne],"[Australia, Sri Lanka]",field,Sri Lanka,"[MD Martell, P Wilson]",Melbourne Cricket Ground,1,NaN
1,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-19],T20,Sri Lanka,20,[DAS Gunaratne],"[Australia, Sri Lanka]",field,Sri Lanka,"[SD Fry, SJ Nogajski]","Simonds Stadium, South Geelong",2,Victoria
2,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-22],T20,Australia,20,[A Zampa],"[Australia, Sri Lanka]",field,Sri Lanka,"[MD Martell, P Wilson]",Adelaide Oval,3,NaN
3,"[{'1st innings': {'team': 'Hong Kong', 'delive...",[2016-09-05],T20,Hong Kong,20,NaN,"[Ireland, Hong Kong]",bat,Hong Kong,"[R Black, AJ Neill]","Bready Cricket Club, Magheramason",4,Londonderry
4,"[{'1st innings': {'team': 'Zimbabwe', 'deliver...",[2016-06-18],T20,Zimbabwe,20,[E Chigumbura],"[Zimbabwe, India]",field,India,"[TJ Matibiri, RB Tiffin]",Harare Sports Club,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,"[{'1st innings': {'team': 'Sri Lanka', 'delive...",[2016-03-04],T20,Pakistan,20,[Umar Akmal],"[Pakistan, Sri Lanka]",field,Pakistan,"[AK Chaudhary, Enamul Haque]",Shere Bangla National Stadium,1418,Mirpur
1418,"[{'1st innings': {'team': 'Bangladesh', 'deliv...",[2016-03-06],T20,India,20,[S Dhawan],"[Bangladesh, India]",field,India,"[RSA Palliyaguruge, Shozab Raza]",Shere Bangla National Stadium,1419,Mirpur
1419,"[{'1st innings': {'team': 'Netherlands', 'deli...",[2016-02-03],T20,Netherlands,20,[Mudassar Bukhari],"[United Arab Emirates, Netherlands]",field,United Arab Emirates,"[CK Nandan, Sarika Prasad]",ICC Academy,1420,Dubai
1420,"[{'1st innings': {'team': 'Australia', 'delive...",[2016-09-06],T20,Australia,20,[GJ Maxwell],"[Sri Lanka, Australia]",field,Sri Lanka,"[REJ Martinesz, RR Wimalasiri]",Pallekele International Cricket Stadium,1421,NaN


In [13]:
final_df['info.match_type'].value_counts()

info.match_type
T20    956
Name: count, dtype: int64

In [14]:
final_df['info.overs'].value_counts()

info.overs
20    953
50      3
Name: count, dtype: int64

In [15]:
final_df=final_df[final_df['info.overs']==20]
final_df.drop(columns=['info.overs','info.match_type'],inplace=True)
final_df

,innings,info.dates,info.outcome.winner,info.player_of_match,info.teams,info.toss.decision,info.toss.winner,info.umpires,info.venue,match_id,info.city
0,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-17],Sri Lanka,[DAS Gunaratne],"[Australia, Sri Lanka]",field,Sri Lanka,"[MD Martell, P Wilson]",Melbourne Cricket Ground,1,NaN
1,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-19],Sri Lanka,[DAS Gunaratne],"[Australia, Sri Lanka]",field,Sri Lanka,"[SD Fry, SJ Nogajski]","Simonds Stadium, South Geelong",2,Victoria
2,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-22],Australia,[A Zampa],"[Australia, Sri Lanka]",field,Sri Lanka,"[MD Martell, P Wilson]",Adelaide Oval,3,NaN
3,"[{'1st innings': {'team': 'Hong Kong', 'delive...",[2016-09-05],Hong Kong,NaN,"[Ireland, Hong Kong]",bat,Hong Kong,"[R Black, AJ Neill]","Bready Cricket Club, Magheramason",4,Londonderry
4,"[{'1st innings': {'team': 'Zimbabwe', 'deliver...",[2016-06-18],Zimbabwe,[E Chigumbura],"[Zimbabwe, India]",field,India,"[TJ Matibiri, RB Tiffin]",Harare Sports Club,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1417,"[{'1st innings': {'team': 'Sri Lanka', 'delive...",[2016-03-04],Pakistan,[Umar Akmal],"[Pakistan, Sri Lanka]",field,Pakistan,"[AK Chaudhary, Enamul Haque]",Shere Bangla National Stadium,1418,Mirpur
1418,"[{'1st innings': {'team': 'Bangladesh', 'deliv...",[2016-03-06],India,[S Dhawan],"[Bangladesh, India]",field,India,"[RSA Palliyaguruge, Shozab Raza]",Shere Bangla National Stadium,1419,Mirpur
1419,"[{'1st innings': {'team': 'Netherlands', 'deli...",[2016-02-03],Netherlands,[Mudassar Bukhari],"[United Arab Emirates, Netherlands]",field,United Arab Emirates,"[CK Nandan, Sarika Prasad]",ICC Academy,1420,Dubai
1420,"[{'1st innings': {'team': 'Australia', 'delive...",[2016-09-06],Australia,[GJ Maxwell],"[Sri Lanka, Australia]",field,Sri Lanka,"[REJ Martinesz, RR Wimalasiri]",Pallekele International Cricket Stadium,1421,NaN


In [16]:
matches=final_df

In [17]:
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

[{0.1: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.2: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.3: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 1, 'extras': 0, 'total': 1}}},
 {0.4: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 2, 'extras': 0, 'total': 2}}},
 {0.5: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.6: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 3, 'extras': 0, 'total': 3}}},
 {1.1: {'batsman': 'M Klinger',
   'bowler': 'KMDN Kulasekara',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 0, 'extras': 0, 'total': 

In [18]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
        })
    delivery_df = pd.concat([delivery_df,loop_df])

In [19]:
delivery_df

,match_id,teams,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue
0,2,"[Australia, Sri Lanka]",Australia,0.1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
1,2,"[Australia, Sri Lanka]",Australia,0.2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
2,2,"[Australia, Sri Lanka]",Australia,0.3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground
3,2,"[Australia, Sri Lanka]",Australia,0.4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground
4,2,"[Australia, Sri Lanka]",Australia,0.5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...,...,...
121,954,"[Sri Lanka, Australia]",Sri Lanka,19.3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium
122,954,"[Sri Lanka, Australia]",Sri Lanka,19.4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium
123,954,"[Sri Lanka, Australia]",Sri Lanka,19.5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium
124,954,"[Sri Lanka, Australia]",Sri Lanka,19.6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium


In [20]:
def bowl(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team


In [21]:
delivery_df['bowling_team']=delivery_df.apply(bowl,axis=1)

In [22]:
delivery_df

,match_id,teams,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,2,"[Australia, Sri Lanka]",Australia,0.1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
1,2,"[Australia, Sri Lanka]",Australia,0.2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
2,2,"[Australia, Sri Lanka]",Australia,0.3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground,Sri Lanka
3,2,"[Australia, Sri Lanka]",Australia,0.4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground,Sri Lanka
4,2,"[Australia, Sri Lanka]",Australia,0.5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
...,...,...,...,...,...,...,...,...,...,...,...
121,954,"[Sri Lanka, Australia]",Sri Lanka,19.3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium,Australia
122,954,"[Sri Lanka, Australia]",Sri Lanka,19.4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium,Australia
123,954,"[Sri Lanka, Australia]",Sri Lanka,19.5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium,Australia
124,954,"[Sri Lanka, Australia]",Sri Lanka,19.6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium,Australia


In [23]:
delivery_df['batting_team'].unique()

array(['Australia', 'Hong Kong', 'Zimbabwe', 'India', 'Bangladesh',
       'New Zealand', 'South Africa', 'England', 'West Indies', 'Ireland',
       'Afghanistan', 'Pakistan', 'United Arab Emirates', 'Scotland',
       'Oman', 'Papua New Guinea', 'Sri Lanka', 'Netherlands', 'Nepal',
       'Vanuatu', 'Philippines', 'United States of America', 'Germany',
       'Ghana', 'Uganda', 'Kenya', 'Namibia', 'Nigeria', 'Botswana',
       'Guernsey', 'Denmark', 'Jersey', 'Italy', 'Norway', 'Thailand',
       'Malaysia', 'Maldives', 'Singapore', 'Kuwait', 'Bermuda', 'Canada',
       'Cayman Islands', 'Portugal', 'Gibraltar', 'Spain', 'Bhutan',
       'Qatar', 'Iran', 'Belgium', 'Isle of Man', 'Bulgaria', 'Romania'],
      dtype=object)

In [24]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]

In [25]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]
delivery_df

,match_id,teams,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
0,2,"[Australia, Sri Lanka]",Australia,0.1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
1,2,"[Australia, Sri Lanka]",Australia,0.2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
2,2,"[Australia, Sri Lanka]",Australia,0.3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground,Sri Lanka
3,2,"[Australia, Sri Lanka]",Australia,0.4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground,Sri Lanka
4,2,"[Australia, Sri Lanka]",Australia,0.5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground,Sri Lanka
...,...,...,...,...,...,...,...,...,...,...,...
121,954,"[Sri Lanka, Australia]",Sri Lanka,19.3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium,Australia
122,954,"[Sri Lanka, Australia]",Sri Lanka,19.4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium,Australia
123,954,"[Sri Lanka, Australia]",Sri Lanka,19.5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium,Australia
124,954,"[Sri Lanka, Australia]",Sri Lanka,19.6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium,Australia


In [26]:
df = delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]


In [27]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo,R Premadasa Stadium
122,954,Sri Lanka,Australia,19.4,0,0,Colombo,R Premadasa Stadium
123,954,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,R Premadasa Stadium
124,954,Sri Lanka,Australia,19.6,2,0,Colombo,R Premadasa Stadium


In [28]:
import pickle

In [29]:
pickle.dump(df,open('dataset_level2.pkl','wb'))

In [30]:

df= pickle.load(open('dataset_level2.pkl','rb'))

In [31]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo,R Premadasa Stadium
122,954,Sri Lanka,Australia,19.4,0,0,Colombo,R Premadasa Stadium
123,954,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,R Premadasa Stadium
124,954,Sri Lanka,Australia,19.6,2,0,Colombo,R Premadasa Stadium


**we want batting team, bowling team,current scire, wickets left, crr, city ,balls left, last five over   these columns**

**feature extraction**

In [32]:
df.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8549
venue                  0
dtype: int64

**venues of the cities missing**

In [33]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       627
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

**if cities are missiing then extract first name of the their venue and put in cities**

In [34]:
cities=np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

In [35]:
df['city']=cities

In [36]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [37]:
df.drop(columns=['venue'],inplace=True)

**now we have batting team,bowling team,and city**

In [38]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo
122,954,Sri Lanka,Australia,19.4,0,0,Colombo
123,954,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
124,954,Sri Lanka,Australia,19.6,2,0,Colombo


In [39]:
df['city'].value_counts()

city
Colombo          4086
Mirpur           3420
Dubai            3092
Johannesburg     2962
Auckland         2409
                 ... 
Victoria          123
Potchefstroom     122
Dharamsala        122
Ahmedabad         121
Carrara            64
Name: count, Length: 86, dtype: int64

**we will only take those cities where min 600 balls were thrown**

In [40]:
df['city'].value_counts()[df['city'].value_counts()>600]

city
Colombo            4086
Mirpur             3420
Dubai              3092
Johannesburg       2962
Auckland           2409
Cape Town          2239
London             2106
Pallekele          2066
Barbados           2050
Melbourne          1574
Durban             1520
St Lucia           1487
Sydney             1457
Lauderhill         1238
Wellington         1227
Hamilton           1219
Centurion          1189
Manchester         1144
Abu Dhabi          1099
Mumbai              995
Nottingham          991
Mount Maunganui     938
Chittagong          868
Kolkata             844
Lahore              748
Delhi               739
Nagpur              735
Southampton         734
Cardiff             730
Chandigarh          627
Adelaide            623
Bangalore           620
St Kitts            618
Christchurch        614
Trinidad            611
Name: count, dtype: int64

In [41]:
df['city'].value_counts()[df['city'].value_counts()>600].sum()

np.int64(49619)

In [42]:
eligible_cities=df['city'].value_counts()[df['city'].value_counts()>600].index.tolist()

In [43]:
df=df[df['city'].isin(eligible_cities)]

In [44]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo
122,954,Sri Lanka,Australia,19.4,0,0,Colombo
123,954,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
124,954,Sri Lanka,Australia,19.6,2,0,Colombo


****

**Now finding current score**

In [45]:
df['current_score']=df['cumulative_runs'] = df.groupby('match_id')['runs'].cumsum()

In [46]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,cumulative_runs
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,1
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,3
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,3
...,...,...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo,125,125
122,954,Sri Lanka,Australia,19.4,0,0,Colombo,125,125
123,954,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,125
124,954,Sri Lanka,Australia,19.6,2,0,Colombo,127,127


**finding Balls left**

**over abd ball split using dot**

In [47]:
df['over']=df['ball'].apply(lambda x: str(x).split(".")[0])
df['ball_no']=df['ball'].apply(lambda x:str(x).split(".")[1])
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,cumulative_runs,over,ball_no
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,0,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,0,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,1,0,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,3,0,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo,125,125,19,3
122,954,Sri Lanka,Australia,19.4,0,0,Colombo,125,125,19,4
123,954,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,125,19,5
124,954,Sri Lanka,Australia,19.6,2,0,Colombo,127,127,19,6


In [48]:
df['balls_bowled']= (df['over'].astype('int')*6) + df['ball_no'].astype('int')

In [49]:
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,cumulative_runs,over,ball_no,balls_bowled,balls_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,0,1,1,119
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,0,2,2,118
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,1,0,3,3,117
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,3,0,4,4,116
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,3,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo,125,125,19,3,117,3
122,954,Sri Lanka,Australia,19.4,0,0,Colombo,125,125,19,4,118,2
123,954,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,125,19,5,119,1
124,954,Sri Lanka,Australia,19.6,2,0,Colombo,127,127,19,6,120,0


**Calculating Wickets left**

**Replace player name with 1 from player dismissed column **

In [50]:
df['player_dismissed']= df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed']= df['player_dismissed'].astype('int')
df['cumulative_player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left']= 10-df['player_dismissed']
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,cumulative_runs,over,ball_no,balls_bowled,balls_left,cumulative_player_dismissed,wickets_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,0,1,1,119,0,10
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,0,2,2,118,0,10
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,1,0,3,3,117,0,10
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,3,0,4,4,116,0,10
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,3,0,5,5,115,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo,125,125,19,3,117,3,8,10
122,954,Sri Lanka,Australia,19.4,0,0,Colombo,125,125,19,4,118,2,8,10
123,954,Sri Lanka,Australia,19.5,0,1,Colombo,125,125,19,5,119,1,9,9
124,954,Sri Lanka,Australia,19.6,2,0,Colombo,127,127,19,6,120,0,9,10


**Calulatig crr**

crr=current score *6 + balls

In [51]:
df['crr']= (df['current_score']*6)/df['balls_bowled']
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,cumulative_runs,over,ball_no,balls_bowled,balls_left,cumulative_player_dismissed,wickets_left,crr
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,0,1,1,119,0,10,0.000000
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,0,2,2,118,0,10,0.000000
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,1,0,3,3,117,0,10,2.000000
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,3,0,4,4,116,0,10,4.500000
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,3,0,5,5,115,0,10,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo,125,125,19,3,117,3,8,10,6.410256
122,954,Sri Lanka,Australia,19.4,0,0,Colombo,125,125,19,4,118,2,8,10,6.355932
123,954,Sri Lanka,Australia,19.5,0,1,Colombo,125,125,19,5,119,1,9,9,6.302521
124,954,Sri Lanka,Australia,19.6,2,0,Colombo,127,127,19,6,120,0,9,10,6.350000


**Calculating last five overs**

In [52]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []
for id in match_ids:
    group = groups.get_group(id)
    # Ensure only numeric data is used
    numeric_data = group['runs'].astype(float)
    last_five.extend(numeric_data.rolling(window=30).sum().values.tolist())

In [53]:
df['last_five']= last_five

In [54]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,cumulative_runs,over,ball_no,balls_bowled,balls_left,cumulative_player_dismissed,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,0,1,1,119,0,10,0.000000,NaN
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,0,2,2,118,0,10,0.000000,NaN
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,1,0,3,3,117,0,10,2.000000,NaN
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,3,0,4,4,116,0,10,4.500000,NaN
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,3,0,5,5,115,0,10,3.600000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,954,Sri Lanka,Australia,19.3,1,0,Colombo,125,125,19,3,117,3,8,10,6.410256,32.0
122,954,Sri Lanka,Australia,19.4,0,0,Colombo,125,125,19,4,118,2,8,10,6.355932,32.0
123,954,Sri Lanka,Australia,19.5,0,1,Colombo,125,125,19,5,119,1,9,9,6.302521,32.0
124,954,Sri Lanka,Australia,19.6,2,0,Colombo,127,127,19,6,120,0,9,10,6.350000,33.0


In [55]:
final_df1 = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [56]:
final_df1=final_df1[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [57]:
final_df1.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_five        11821
runs_x               0
dtype: int64

In [58]:
final_df1.dropna(inplace=True)

In [59]:
final_df1.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

> Now the final dataset is ready and we have created all feature as required for predicting the score in a cricket macth now training the data on a model and making the predictions

shuffle for no bias

In [60]:
final_df1 = final_df1.sample(final_df1.shape[0])

In [61]:
final_df1

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
11127,India,New Zealand,Hamilton,71,81,10,10.923077,59.0,179
1836,New Zealand,Australia,Sydney,90,22,10,5.510204,32.0,117
45950,New Zealand,India,Nagpur,89,27,10,5.741935,29.0,126
19448,England,South Africa,Nottingham,34,70,10,4.080000,19.0,111
2236,New Zealand,Australia,Auckland,226,4,10,11.689655,42.0,243
...,...,...,...,...,...,...,...,...,...
31088,Bangladesh,Pakistan,Pallekele,152,13,10,8.523364,34.0,175
17865,Australia,England,Manchester,48,78,10,6.857143,36.0,145
31259,West Indies,England,Pallekele,45,86,10,7.941176,43.0,179
31040,Bangladesh,Pakistan,Pallekele,90,60,10,9.000000,41.0,175


In [62]:
X = final_df1.drop(columns=['runs_x'])
y = final_df1['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

**one hot encoding for first three columns**

In [64]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [65]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [66]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.976578950881958
2.161664445396033


In [67]:
import pickle

# Save the trained model as a .pkl file
with open('pipe.pkl', 'wb') as f:
    pickle.dump(pipe, f)


In [68]:
teams

['Australia',
 'India',
 'Bangladesh',
 'New Zealand',
 'South Africa',
 'England',
 'West Indies',
 'Afghanistan',
 'Pakistan',
 'Sri Lanka']